In [1]:
import requests
import json

## Autenticação
### Fluxo de Retry
- Implementação do fluxo de re-autenticação

- Principais Tipos de Autenticação

1. Baerer Token
2. OAuth
3. API Key

In [2]:
def autenticar(login, senha):
    login_response = requests.post(
        url="https://dummyjson.com/auth/login",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "username": login,
            "password": senha,
            "expiresInMins": 1
        })
    )
    
    return login_response.json()['token']

access_token = autenticar(login="kminchelle", senha="0lelplR")

In [ ]:
url_base = "https://dummyjson.com/auth/"

authorization_header = {
    "Authorization": f"Bearer {access_token}", 
    "Content-Type": "application/json"
}

In [ ]:
## Caso de Sucesso com Authenticação

authenticated_products_response = requests.get(
    url=f"{url_base}/products",
    headers=authorization_header
)

print(f"Status Code: {authenticated_products_response.status_code}")
print(f"Content: {authenticated_products_response.text}")

In [ ]:
authorization_header_errado = {
    "Authorization": f"Bearer 123654646156475656498789656", 
    "Content-Type": "application/json"
}

authenticated_products_response = requests.get(
    url=f"{url_base}/products",
    headers=authorization_header_errado
)

print(f"Status Code: {authenticated_products_response.status_code}")

In [ ]:
### Implementação do Fluxo de Re-autenticação

In [ ]:
def constroi_authentication_headers():
    access_token = autenticar(login="kminchelle", senha="0lelplR")
    
    return {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

In [ ]:
def gerencia_requisicao_authenticada(url: str, authorization_header: dict, max_retries: int, current_retry: int =0):

    response = requests.get(
        url=url,
        headers=authorization_header
    )
    
    try:
        response_content = response.json()
        
        if response.status_code == 401:
            if current_retry < max_retries:
                if response_content['name'] == "TokenExpiredError":
                    return gerencia_requisicao_authenticada(
                        url=url,
                        authorization_header=constroi_authentication_headers(),
                        max_retries=max_retries,
                        current_retry=current_retry+1
                    )
            else:
                print(f"Não foi possível recuperar a requisição após o código. Código: {response.status_code}")
          
        return response_content
    except Exception as ex:
        print(f"Fluxo de exceção. Detalhe: {ex}")

In [ ]:
gerencia_requisicao_authenticada(url=f"{url_base}/products", authorization_header=authorization_header, max_retries=3)

### Acessando apenas os recursos desejados em uma API
Filtrando recursis por Request Path

In [ ]:
def constroi_authentication_headers():
    access_token = autenticar(login="kminchelle", senha="0lelplR")
    
    return {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

In [ ]:
query_reponse = requests.get(url="https://dummyjson.com/auth/products/search?q=phone", headers=constroi_authentication_headers())
query_reponse.json()

In [ ]:
## Trabalhando com Paginação
## Parametros de Paginação me ajudam e definir qual intervalo de dados eu desejo consumir
## Onde é util:
## 5.000.000
## Lambda 1KB
## 5.000.000 - API pode demorar muito pra responder, a API pode estar programada pra não responder tudo isso, ou programa pode não ter
## Capacidade de processar tudo isso

## 0  Skip
## 10 limit
## 100 total

## 10 skip 
## 10 limit
## 100 total

## 20 skip 
## 10 limit
## 100 total

## 90 skip 
## 10 limit
## 100 total



response_paginado = requests.get("https://dummyjson.com/auth/products?limit=10&skip=10&select=title,price", headers=constroi_authentication_headers())

response_paginado.json()